# Data Cleaning and Wrangling
*Loading, cleaning, tidying and reshaping data with R.*

*Note: The tutorial series works alot with tidyr, dplyr, ggplot and similar packages.
All of these libraries provide a concise and clear way to execute data reshaping and visualization in R beyond the vanilla approach.*

If you need information about a function or package simply type ?*function_name*

**tidyr:** helps tidying data frames

**dplyr:** is a grammer for data manipulation

**ggplot2:** is a grammar for graphics to declaratively declare plots

In [10]:
library(tidyr)
library(dplyr, warn.conflicts = F)
library(ggplot2)

In [11]:
?dplyr

## Import Data
R allows you to import data from csv, json but also from many other file formats like hdf5 or databases.
Start with loading you data into a minimal set of logical related data frames and transform them into dplyr tables.

In [12]:
df <- read.csv('datasets/student-mat.csv', sep = ';') %>%
    tbl_df
df %>% head

school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,10,15,15,15


### Conclusion on Importing
* R can access various data sources and the first step is to get a minimal set of data frames.
* Sometimes R automatically converts strings to factors - *be aware*.
* Encapsulate your loading logic in its own script to avoid change propagation.

## Tidy Data

[Tidy data](ftp://cran.r-project.org/pub/R/web/packages/tidyr/vignettes/tidy-data.html) describes a set of principles to organize data such that the follow up analysis is simplified.
It is focused around data frames in which *columns* describe the variable and
*rows* the *observations*.
A variable (a column) could be the name of a person and every entry within the column describes a specific name e.g., John Doe.

This is similar to database where tidy data would be a table in Codd's 3rd normal form but reframed in a statistical fashion:
1. Each variable forms a column,
2. each observation forms a row,
3. each type of observational unit forms a table.


The five most common problems with data and their respective data sets are:
1. Column headers are values, not variable names.
2. Multiple variables are stored in one column.
3. Variables are stored in both rows and columns.
4. Multiple types of observational units are stored in the same table.
5. A single observation unit is stored in multiple tables.

Hadely provides an example for all these issues on the link above, so browse through.

### Conventions

In addition to these common tidy principles it is highly recommended to use common software development best practices for naming variables but also for their respective values.

* [Google R](https://google.github.io/styleguide/Rguide.xml)
* [Hadley Wickham](http://adv-r.had.co.nz/Style.html)


#### Rename Variable
dplyr provides the *rename*-function that renames columns.

In [13]:
df <- df %>%
    rename(Sex = sex,
           Age = age,
           School = school,
           HomeType = address,
           ParentStatus = Pstatus,
           EducationMother = Medu,
           JobMother = Mjob,
           EducationFather = Fedu,
           JobFather = Fjob,
           Guardian = guardian,
           FamilySize = famsize,
           FamilyRelationship = famrel,
           SchoolChoiceReason = reason,
           TravelTime = traveltime,
           StudyTime = studytime,
           ClassFailed = failures,
           EducationalSchoolSupport = schoolsup,
           EducationalFamilySupport = famsup,
           ExtraCurricularActivities = activities,
           ExtraPaidClass = paid,
           InternetAccess = internet,
           AttendedNurserySchool = nursery,
           TargetsHigherEducation = higher,
           RelationshipStatus = romantic,
           LeisureTime = freetime,
           SocialInteractionIntensity = goout,
           AlcoholConsumptionWeekend = Walc,
           AlcoholConsumptionWorkday = Dalc,
           HealthStatus = health,
           SchoolAbsences = absences,
           FirstPeriodGrade = G1,
           SecondPeriodGrade = G2,
           FinalGrade = G3) 

df %>% head

School,Sex,Age,HomeType,FamilySize,ParentStatus,EducationMother,EducationFather,JobMother,JobFather,...,FamilyRelationship,LeisureTime,SocialInteractionIntensity,AlcoholConsumptionWorkday,AlcoholConsumptionWeekend,HealthStatus,SchoolAbsences,FirstPeriodGrade,SecondPeriodGrade,FinalGrade
GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,10,15,15,15


#### Recode Values
dplyr provides the *recode* function which is a disguiste *switch* statement.
Note that the assignment is reverse to the *select* or *rename* function.

In [14]:
RecodeEducation <- function(x) recode(x, `0` = 'None', `1` = 'Primary', `2` = 'PrimaryExtended', `3` = 'SecondaryExtended', `4` = 'Higher') 
RecodeJob <- function(x) recode(x, teacher = 'Education', services = 'Services', at_home = 'Home', other = 'Other', health = 'Health')
RecodeBinary <- function(x) recode(x, yes = 'Yes', no = 'No')
RecodeLikert <- function(x) recode(x, `1` = 'VeryLow', `2` = 'Low', `3` = 'Medium', `4` = 'High', `5` = 'VeryHigh')
    
df <- df %>%
    mutate(Sex = recode(Sex, F = 'Female', M = 'Male'),
           School = recode(School, GP = 'GabrielPereira', MS = 'MousinhoDaSilveira'),
           HomeType = recode(HomeType, U = 'Urban', R = 'Rural'),
           ParentStatus = recode(ParentStatus, T = 'Together', A = 'Apart'),
           EducationMother = RecodeEducation(EducationMother),
           JobMother = RecodeJob(JobMother),
           EducationFather = RecodeEducation(EducationFather),
           JobFather = RecodeJob(JobFather),
           Guardian = recode(Guardian, mother = 'Mother', father = 'Father', other = 'Other'),
           FamilySize = recode(FamilySize, GT3 = 'Large', LE3 = 'Small'),
           FamilyRelationship = recode(FamilyRelationship, `1` = 'VeryBad', `2` = 'Bad', `3` = 'Ok', `4` = 'Good', `5` = 'VeryGood'),
           SchoolChoiceReason = recode(SchoolChoiceReason, course = 'CoursePreference', other = 'Other', home = 'HomeProximity', reputation = 'Reputation'),
           TravelTime = recode(TravelTime, `1` = 'x < 15', `2` = '15 <= x < 30', `3` = '30 <= x < 60', `4` = 'x >= 60'),
           StudyTime = recode(StudyTime, `1` = 'x < 120', `2` = '120 <= x < 300', `3` = '300 <= x < 600', `4` = 'x >= 600')) %>%
    mutate_at(vars(EducationMother, EducationFather), .funs = RecodeEducation) %>%
    mutate_at(vars(JobMother, JobFather), .funs = RecodeJob) %>%
    mutate_at(vars(EducationalFamilySupport, 
                   EducationalSchoolSupport, 
                   ExtraCurricularActivities, 
                   ExtraPaidClass, 
                   InternetAccess, 
                   AttendedNurserySchool, 
                   TargetsHigherEducation, 
                   RelationshipStatus),
              .funs = RecodeBinary) %>%
    mutate_at(vars(LeisureTime,
                   SocialInteractionIntensity,
                   AlcoholConsumptionWeekend,
                   AlcoholConsumptionWorkday,
                   HealthStatus),
              .funs = RecodeLikert)
    
df %>% head

School,Sex,Age,HomeType,FamilySize,ParentStatus,EducationMother,EducationFather,JobMother,JobFather,...,FamilyRelationship,LeisureTime,SocialInteractionIntensity,AlcoholConsumptionWorkday,AlcoholConsumptionWeekend,HealthStatus,SchoolAbsences,FirstPeriodGrade,SecondPeriodGrade,FinalGrade
GabrielPereira,Female,18,Urban,Large,Apart,Higher,Higher,Home,Education,...,Good,Medium,High,VeryLow,VeryLow,Medium,6,5,6,6
GabrielPereira,Female,17,Urban,Large,Together,Primary,Primary,Home,Other,...,VeryGood,Medium,Medium,VeryLow,VeryLow,Medium,4,5,5,6
GabrielPereira,Female,15,Urban,Small,Together,Primary,Primary,Home,Other,...,Good,Medium,Low,Low,Medium,Medium,10,7,8,10
GabrielPereira,Female,15,Urban,Large,Together,Higher,PrimaryExtended,Health,Services,...,Ok,Low,Low,VeryLow,VeryLow,VeryHigh,2,15,14,15
GabrielPereira,Female,16,Urban,Large,Together,SecondaryExtended,SecondaryExtended,Other,Other,...,Good,Medium,Low,VeryLow,Low,VeryHigh,4,6,10,10
GabrielPereira,Male,16,Urban,Small,Together,Higher,SecondaryExtended,Services,Other,...,VeryGood,High,Low,VeryLow,Low,VeryHigh,10,15,15,15


## Inspect data
Check the basic characteristics of you variables via the summary function.

* Are the variables names correct?
* Is the type correct?
* Is the basic distribution of your categorical data as expected or did you omit, reverse, lost values?
* Is the basic distribution of you interval data as expected or are the maxima, mean, quantiles off?
* Are all variables of the correct type (factor or characters)?
* Are all variables really variables or are they actual values?
* Do you have a unique identifier for each observation?

In [15]:
summary(df)

                School        Sex           Age        HomeType   FamilySize 
 GabrielPereira    :349   Female:208   Min.   :15.0   Rural: 88   Large:281  
 MousinhoDaSilveira: 46   Male  :187   1st Qu.:16.0   Urban:307   Small:114  
                                       Median :17.0                          
                                       Mean   :16.7                          
                                       3rd Qu.:18.0                          
                                       Max.   :22.0                          
   ParentStatus EducationMother    EducationFather        JobMother  
 Apart   : 41   Length:395         Length:395         Home     : 59  
 Together:354   Class :character   Class :character   Health   : 34  
                Mode  :character   Mode  :character   Other    :141  
                                                      Services :103  
                                                      Education: 58  
                                  

#### Identifiers
Use unique identifiers for observations to join different tables but also to keep track of the different observation as you reshape it form wide to long format because of visualization purposes.

In [16]:
df <- df %>%
    mutate(Id = row_number()) %>%
    select(Id, everything())
df %>% head

Id,School,Sex,Age,HomeType,FamilySize,ParentStatus,EducationMother,EducationFather,JobMother,...,FamilyRelationship,LeisureTime,SocialInteractionIntensity,AlcoholConsumptionWorkday,AlcoholConsumptionWeekend,HealthStatus,SchoolAbsences,FirstPeriodGrade,SecondPeriodGrade,FinalGrade
1,GabrielPereira,Female,18,Urban,Large,Apart,Higher,Higher,Home,...,Good,Medium,High,VeryLow,VeryLow,Medium,6,5,6,6
2,GabrielPereira,Female,17,Urban,Large,Together,Primary,Primary,Home,...,VeryGood,Medium,Medium,VeryLow,VeryLow,Medium,4,5,5,6
3,GabrielPereira,Female,15,Urban,Small,Together,Primary,Primary,Home,...,Good,Medium,Low,Low,Medium,Medium,10,7,8,10
4,GabrielPereira,Female,15,Urban,Large,Together,Higher,PrimaryExtended,Health,...,Ok,Low,Low,VeryLow,VeryLow,VeryHigh,2,15,14,15
5,GabrielPereira,Female,16,Urban,Large,Together,SecondaryExtended,SecondaryExtended,Other,...,Good,Medium,Low,VeryLow,Low,VeryHigh,4,6,10,10
6,GabrielPereira,Male,16,Urban,Small,Together,Higher,SecondaryExtended,Services,...,VeryGood,High,Low,VeryLow,Low,VeryHigh,10,15,15,15


#### Typical Reshape
Now that we actually know that G1-G3 are just grades we might want to consolidate the three columns into on categorical column describing the type of grade (Frist, Second, Final) and one column that actually contains the grade.

In [17]:
# use tidyr to collect multiple columns into two columns
df <- df %>% 
    gather(key = GradeName, 
           value = Grade, 
           FirstPeriodGrade, SecondPeriodGrade, FinalGrade)
df %>% head

Id,School,Sex,Age,HomeType,FamilySize,ParentStatus,EducationMother,EducationFather,JobMother,...,RelationshipStatus,FamilyRelationship,LeisureTime,SocialInteractionIntensity,AlcoholConsumptionWorkday,AlcoholConsumptionWeekend,HealthStatus,SchoolAbsences,GradeName,Grade
1,GabrielPereira,Female,18,Urban,Large,Apart,Higher,Higher,Home,...,No,Good,Medium,High,VeryLow,VeryLow,Medium,6,FirstPeriodGrade,5
2,GabrielPereira,Female,17,Urban,Large,Together,Primary,Primary,Home,...,No,VeryGood,Medium,Medium,VeryLow,VeryLow,Medium,4,FirstPeriodGrade,5
3,GabrielPereira,Female,15,Urban,Small,Together,Primary,Primary,Home,...,No,Good,Medium,Low,Low,Medium,Medium,10,FirstPeriodGrade,7
4,GabrielPereira,Female,15,Urban,Large,Together,Higher,PrimaryExtended,Health,...,Yes,Ok,Low,Low,VeryLow,VeryLow,VeryHigh,2,FirstPeriodGrade,15
5,GabrielPereira,Female,16,Urban,Large,Together,SecondaryExtended,SecondaryExtended,Other,...,No,Good,Medium,Low,VeryLow,Low,VeryHigh,4,FirstPeriodGrade,6
6,GabrielPereira,Male,16,Urban,Small,Together,Higher,SecondaryExtended,Services,...,No,VeryGood,High,Low,VeryLow,Low,VeryHigh,10,FirstPeriodGrade,15


### Conclusion on Tidying Data

* Columns are variables and rows are observations
* Each dataframe captures one concept
* Be consistent in naming and use style guides
* Inspect you data frame via summary but also the clean values 
* Encapsulate your tidying logic in its own script to avoid change propagation


## Working with data frames

* How can i compute summary statistics?
* How can i compute new columns?
* How can i join tabes?
* How can i order data frames?

What is average grade of student "1"?

In [18]:
# observe effect of adding column Id
df %>%
    arrange(Id) %>% 
    head

Id,School,Sex,Age,HomeType,FamilySize,ParentStatus,EducationMother,EducationFather,JobMother,...,RelationshipStatus,FamilyRelationship,LeisureTime,SocialInteractionIntensity,AlcoholConsumptionWorkday,AlcoholConsumptionWeekend,HealthStatus,SchoolAbsences,GradeName,Grade
1,GabrielPereira,Female,18,Urban,Large,Apart,Higher,Higher,Home,...,No,Good,Medium,High,VeryLow,VeryLow,Medium,6,FirstPeriodGrade,5
1,GabrielPereira,Female,18,Urban,Large,Apart,Higher,Higher,Home,...,No,Good,Medium,High,VeryLow,VeryLow,Medium,6,SecondPeriodGrade,6
1,GabrielPereira,Female,18,Urban,Large,Apart,Higher,Higher,Home,...,No,Good,Medium,High,VeryLow,VeryLow,Medium,6,FinalGrade,6
2,GabrielPereira,Female,17,Urban,Large,Together,Primary,Primary,Home,...,No,VeryGood,Medium,Medium,VeryLow,VeryLow,Medium,4,FirstPeriodGrade,5
2,GabrielPereira,Female,17,Urban,Large,Together,Primary,Primary,Home,...,No,VeryGood,Medium,Medium,VeryLow,VeryLow,Medium,4,SecondPeriodGrade,5
2,GabrielPereira,Female,17,Urban,Large,Together,Primary,Primary,Home,...,No,VeryGood,Medium,Medium,VeryLow,VeryLow,Medium,4,FinalGrade,6


In [19]:
# group by student id and summarise its grade
gradeMean_df <- df %>% 
    group_by(Id) %>%
    summarise(GradeMean = mean(Grade))

gradeMean_df %>%
    head

Id,GradeMean
1,5.666667
2,5.333333
3,8.333333
4,14.666667
5,8.666667
6,15.000000


What are the grades in the Austrian mark system?

In [20]:
scale <- function(x, oldMax, oldMin, newMax, newMin){
  newMin + ((x - oldMin) * (newMax - newMin) / (oldMax - oldMin))  
} 

gradeAustrian_df <- df %>%
    mutate(GradeAustrian = scale(Grade, 
                                 oldMax = 20, oldMin = 0,
                                 newMax = 5, newMin = 1),
           GradeAustrian = 6 - GradeAustrian)
gradeAustrian_df %>%
    head

Id,School,Sex,Age,HomeType,FamilySize,ParentStatus,EducationMother,EducationFather,JobMother,...,FamilyRelationship,LeisureTime,SocialInteractionIntensity,AlcoholConsumptionWorkday,AlcoholConsumptionWeekend,HealthStatus,SchoolAbsences,GradeName,Grade,GradeAustrian
1,GabrielPereira,Female,18,Urban,Large,Apart,Higher,Higher,Home,...,Good,Medium,High,VeryLow,VeryLow,Medium,6,FirstPeriodGrade,5,4.0
2,GabrielPereira,Female,17,Urban,Large,Together,Primary,Primary,Home,...,VeryGood,Medium,Medium,VeryLow,VeryLow,Medium,4,FirstPeriodGrade,5,4.0
3,GabrielPereira,Female,15,Urban,Small,Together,Primary,Primary,Home,...,Good,Medium,Low,Low,Medium,Medium,10,FirstPeriodGrade,7,3.6
4,GabrielPereira,Female,15,Urban,Large,Together,Higher,PrimaryExtended,Health,...,Ok,Low,Low,VeryLow,VeryLow,VeryHigh,2,FirstPeriodGrade,15,2.0
5,GabrielPereira,Female,16,Urban,Large,Together,SecondaryExtended,SecondaryExtended,Other,...,Good,Medium,Low,VeryLow,Low,VeryHigh,4,FirstPeriodGrade,6,3.8
6,GabrielPereira,Male,16,Urban,Small,Together,Higher,SecondaryExtended,Services,...,VeryGood,High,Low,VeryLow,Low,VeryHigh,10,FirstPeriodGrade,15,2.0


Are the boundaries correctly computed?
What are arbitrary max, min and midpoint values to check the conversion?

In [21]:
 gradeAustrian_df %>%
    select(Id, Grade, GradeAustrian) %>%
    filter(Grade == 0 | Grade == 10 | Grade == 20) %>%
    distinct(Grade, .keep_all=TRUE)

Id,Grade,GradeAustrian
11,10,3
131,0,5
48,20,1


How can i add the mean grade to the existing data frame?

In [25]:
# dplyr uses automatically matching columns to join on
# df %>%
#     inner_join(gradeMean_df)

# or if it is only one column simply defined the column
# df %>%
#     inner_join(gradeMean_df, by = 'Id')

# but best define the mapping to avoid mistakes
df %>%
    inner_join(gradeMean_df, by = c('Id' = 'Id')) %>%
    head

Id,School,Sex,Age,HomeType,FamilySize,ParentStatus,EducationMother,EducationFather,JobMother,...,FamilyRelationship,LeisureTime,SocialInteractionIntensity,AlcoholConsumptionWorkday,AlcoholConsumptionWeekend,HealthStatus,SchoolAbsences,GradeName,Grade,GradeMean
1,GabrielPereira,Female,18,Urban,Large,Apart,Higher,Higher,Home,...,Good,Medium,High,VeryLow,VeryLow,Medium,6,FirstPeriodGrade,5,5.666667
2,GabrielPereira,Female,17,Urban,Large,Together,Primary,Primary,Home,...,VeryGood,Medium,Medium,VeryLow,VeryLow,Medium,4,FirstPeriodGrade,5,5.333333
3,GabrielPereira,Female,15,Urban,Small,Together,Primary,Primary,Home,...,Good,Medium,Low,Low,Medium,Medium,10,FirstPeriodGrade,7,8.333333
4,GabrielPereira,Female,15,Urban,Large,Together,Higher,PrimaryExtended,Health,...,Ok,Low,Low,VeryLow,VeryLow,VeryHigh,2,FirstPeriodGrade,15,14.666667
5,GabrielPereira,Female,16,Urban,Large,Together,SecondaryExtended,SecondaryExtended,Other,...,Good,Medium,Low,VeryLow,Low,VeryHigh,4,FirstPeriodGrade,6,8.666667
6,GabrielPereira,Male,16,Urban,Small,Together,Higher,SecondaryExtended,Services,...,VeryGood,High,Low,VeryLow,Low,VeryHigh,10,FirstPeriodGrade,15,15.000000
